# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('ETL_Preparation', engine)
X = df.message
y = df[df.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '...................................................')

related :
                        precision    recall  f1-score   support

               related       0.63      0.37      0.47      1527
               request       0.82      0.93      0.88      4983
                 offer       0.75      0.14      0.23        44

           avg / total       0.78      0.80      0.78      6554
 ...................................................
request :
                        precision    recall  f1-score   support

               related       0.88      0.98      0.93      5416
               request       0.80      0.37      0.51      1138

           avg / total       0.87      0.87      0.85      6554
 ...................................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6527
               request       0.00      0.00      0.00        27

           avg / total       0.99      1.00      0.99      6554
 ...........................

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
pipeline.get_params()

parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 20, 40],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)


In [40]:
#MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  44.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   54.0s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  44.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  45.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 1.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 1.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 1.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40, total= 2.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40 
[CV]  clf_

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 43.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20, 40], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [41]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.21865, std: 0.00327, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 10},
 mean: 0.22444, std: 0.00812, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20},
 mean: 0.22963, std: 0.00630, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 40},
 mean: 0.21412, std: 0.00872, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10},
 mean: 0.21803, std: 0.00257, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 20},
 mean: 0.22749, std: 0.00733, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 40},
 mean: 0.20334, std: 0.00389, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 10},
 mean: 0.21875, std: 0.00838, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 20},
 mean: 0.22688, std: 0.00819, params: {'clf__estimator__min_samples_spli

In [42]:
#finding the best paramesters based on grip search
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 40}


In [43]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [44]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '...................................................')

related :
             precision    recall  f1-score   support

          0       0.73      0.30      0.42      1527
          1       0.81      0.97      0.88      4983
          2       0.53      0.20      0.30        44

avg / total       0.79      0.81      0.77      6554
 ...................................................
request :
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5416
          1       0.87      0.43      0.58      1138

avg / total       0.89      0.89      0.87      6554
 ...................................................
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554
 ...................................................
aid_related :
             precision    recall  f1-score   support

          0       0.76      0.88      0.82      38

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [48]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

#Use pipeline1 which includes StartingVerbEstimator

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline1.fit(X_train, y_train)

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# Print classification report on test data
for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '...................................................')

related :
                        precision    recall  f1-score   support

               related       0.94      0.84      0.88      1576
               request       0.95      0.98      0.96      4936
                 offer       0.97      0.69      0.81        42

           avg / total       0.95      0.95      0.94      6554
 ...................................................
request :
                        precision    recall  f1-score   support

               related       0.96      1.00      0.98      5522
               request       0.97      0.78      0.86      1032

           avg / total       0.96      0.96      0.96      6554
 ...................................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6525
               request       1.00      0.52      0.68        29

           avg / total       1.00      1.00      1.00      6554
 ...........................

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 9. Export your model as a pickle file

In [49]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.